**Task 07: Querying RDF(s)**

In [ ]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")


for subj, pred, obj in g:
  print(subj,pred,obj)



http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Property
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://somewhere#JohnSmith http:/

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [ ]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#")
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")

for s,p,o in g.triples((None, RDFS.subClassOf , ns.Person)):
  print('RDFLib')
  print(s,p,o)



q1 = prepareQuery('''
  SELECT 
    ?Subject
  WHERE { 
    ?Subject (RDFS:subClassOf/RDFS:subClassOf*) ns:Person. 
  }
  ''',
  initNs = { "RDF": RDF,"RDFS": RDFS, "vcard": VCARD, "ns":ns}
)

print('SPARQL')
for r in g.query(q1):
  print(r)

RDFLib
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person
SPARQL
(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [ ]:
from rdflib import XSD

print('RDFLib')

for s,p,o in g.triples((None, RDFS.subClassOf , ns.Person)): # Subclases de persona
  for su, pred, obj in g.triples((None, RDF.type, s)): # Obtengo el sujeto
     print(su)
for s,p,o in g.triples((None, RDF.type, ns.Person)): # Obtengo las personas
    print(s)

 

q2 = prepareQuery('''
  SELECT 
    ?su 
  WHERE { 
    
    {?s RDFS:subClassOf ns:Person.
    ?su RDF:type ?s.}UNION{
    ?su RDF:type ns:Person}
  }
  ''',
  initNs = {"RDF": RDF,"RDFS": RDFS, "vcard": VCARD, "ns":ns}
)


for r in g.query(q2):
  print(r)

RDFLib
http://somewhere#JaneSmith
http://somewhere#JohnSmith
http://somewhere#SaraJones
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [ ]:
print('RDFLib')

for s,p,o in g.triples((None, RDFS.subClassOf , ns.Person)): # Subclases de persona
  for su, pred, obj in g.triples((None, RDF.type, s)): # Obtengo el sujeto/personas (que seran URI de las personas)
     for sujeto, predicado, objeto in g.triples((su, None, None)): # De las personas obtengo su predicado y su valor
       print(sujeto, predicado, objeto)


for s,p,o in g.triples((None, RDF.type, ns.Person)): # Obtengo las personas
    for sujeto, predicado, objeto in g.triples((s, None, None)):
       print(sujeto, predicado, objeto)

 



q3 = prepareQuery('''
  SELECT DISTINCT
    ?s ?p ?o
  WHERE { 
    
    {
      ?s1 RDFS:subClassOf ns:Person.
      ?su RDF:type ?s1.
    }UNION{
      ?su RDF:type ns:Person
    }UNION{
      ?s ?p ?o
    }
  }
  ''',
  initNs = {"RDF": RDF,"RDFS": RDFS, "vcard": VCARD, "ns":ns}
)

print("SPARQL")
for r in g.query(q3):
  print(r)

RDFLib
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewher

In [ ]:
|